# SQL Assignment

In [1]:
import pandas as pd
import sqlite3

In [2]:
conn = sqlite3.connect("Db-IMDB-Assignment.db")

## Sample Code

In [3]:
%%time
# Write your sql query below

query = """
        SELECT TRIM(Movie.title) AS 'Movie_Name'
        FROM Movie
        WHERE Movie.rating < 3
        
        """

q = pd.read_sql_query(query, conn)    
print(q.shape)
q.head()

(85, 1)
CPU times: user 90.2 ms, sys: 14 ms, total: 104 ms
Wall time: 106 ms


,Movie_Name
0,Mastizaade
1,Dragonball Evolution
2,Loveyatri
3,Race 3
4,Gunday


## Q1 --- List all the directors who directed a 'Comedy' movie in a leap year. (You need to check that the genre is 'Comedy’ and year is a leap year) Your query should return director name, the movie name, and the year.

In [14]:
%%time
# Write your sql query below

query = """
        SELECT P.name as "Director Name", M.title as "Movie Name", M.year as "Year"
        FROM Genre G
        LEFT JOIN M_Genre MG ON G.GID = MG.GID
        LEFT JOIN M_Director MD ON MG.MID = MD.MID
        LEFT JOIN Person P ON MD.PID = P.PID
        LEFT JOIN Movie M on MG.MID = M.MID
        WHERE G.Name LIKE '%Comedy%'
        AND (CAST(SUBSTR(M.Year,-4) AS UNSIGNED) % 4 = 0
        AND CAST(SUBSTR(M.Year,-4) AS UNSIGNED) % 100 <> 0
        OR CAST(SUBSTR(M.Year,-4) AS UNSIGNED) % 400 = 0)
        
        """

q1 = pd.read_sql_query(query, conn)    
print(q1.shape)
q1.head()

(232, 3)
CPU times: user 69.1 ms, sys: 2.08 ms, total: 71.1 ms
Wall time: 69.6 ms


,Director Name,Movie Name,Year
0,Prakash Mehra,Hera Pheri,1976
1,Raj Kanwar,Deewana,1992
2,Priyadarshan,Hera Pheri,2000
3,Naresh Kumar,Gora Aur Kala,1972
4,Eeshwar Nivas,My Name Is Anthony Gonsalves,2008


## Q2 --- List the names of all the actors who played in the movie 'Anand' (1971)

In [11]:
%%time
# Write your sql query below

query = """
        SELECT P.Name
        FROM Person P
        WHERE P.PID in
            (SELECT TRIM(MC.PID, " ")
            FROM M_Cast MC
            INNER JOIN Movie M ON M.MID = MC.MID
            WHERE M.title = "Anand"
            AND CAST(SUBSTR(M.Year,-4) AS UNSIGNED) = 1971)
        
        
        """

q2 = pd.read_sql_query(query, conn)    
print(q2.shape)
q2.head()

(17, 1)
CPU times: user 110 ms, sys: 9.75 ms, total: 120 ms
Wall time: 119 ms


,Name
0,Amitabh Bachchan
1,Rajesh Khanna
2,Sumita Sanyal
3,Ramesh Deo
4,Seema Deo


## Q3 --- List all the actors who acted in a film before 1970 and in a film after 1990. (That is: < 1970 and > 1990.)

In [10]:
%%time
# Write your sql query below

query = """
        SELECT P.Name
        FROM Person P
        WHERE P.PID IN
            (SELECT TRIM(MC.PID, " ")
            FROM M_Cast MC
            WHERE MC.MID IN
                (SELECT M.MID
                FROM Movie M
                WHERE CAST(SUBSTR(M.Year,-4) AS UNSIGNED) < 1970))
        AND P.PID IN
            (SELECT TRIM(MC.PID, " ")
            FROM M_Cast MC
            WHERE MC.MID IN
                (SELECT M.MID
                FROM Movie M
                WHERE CAST(SUBSTR(M.Year,-4) AS UNSIGNED) > 1990))
        """


q3 = pd.read_sql_query(query, conn)    
print(q3.shape)
q3.head()

(300, 1)
CPU times: user 125 ms, sys: 5.49 ms, total: 130 ms
Wall time: 129 ms


,Name
0,Rishi Kapoor
1,Amitabh Bachchan
2,Asrani
3,Zohra Sehgal
4,Parikshat Sahni


## Q4 --- List all directors who directed 10 movies or more, in descending order of the number of movies they directed. Return the directors' names and the number of movies each of them directed.

In [7]:
%%time
# Write your sql query below

query = """
        SELECT P.Name, Sub1."Number of Movies"
        FROM Person P
        INNER JOIN (SELECT MD.PID, count(*) AS "Number of Movies"
                    FROM Movie M
                    LEFT JOIN M_Director MD ON MD.MID = M.MID
                    GROUP BY MD.PID
                    HAVING count(*) >= 10) Sub1 
        ON P.PID = Sub1.PID
        ORDER BY Sub1."Number of Movies" desc
        """

q4 = pd.read_sql_query(query, conn)    
print(q4.shape)
q4.head()

(58, 2)
CPU times: user 21.7 ms, sys: 1.93 ms, total: 23.7 ms
Wall time: 22 ms


,Name,Number of Movies
0,David Dhawan,39
1,Mahesh Bhatt,35
2,Ram Gopal Varma,30
3,Priyadarshan,30
4,Vikram Bhatt,29


## Q5.a --- For each year, count the number of movies in that year that had only female actors.

In [ ]:
%%time
# Write your sql query below

query = """



        SELECT CAST(SUBSTR(M.Year,-4) AS UNSIGNED) as 'Year' , count(DISTINCT TRIM(M.MID)) as "Number of Movies"
        FROM Movie M
        WHERE TRIM(M.MID) IN (SELECT TRIM(MC.MID)
                        FROM M_Cast MC
                        WHERE TRIM(MC.PID, " ") IN (SELECT TRIM(P.PID, " ")
                                                    FROM Person P
                                                    WHERE TRIM(P.Gender) LIKE "Female" ))
        AND TRIM(M.MID) NOT IN (SELECT TRIM(MC.MID)
                        FROM M_Cast MC
                        WHERE TRIM(MC.PID, " ") IN (SELECT TRIM(P.PID, " ")
                                                    FROM Person P
                                                    WHERE TRIM(P.Gender) LIKE "Male" OR TRIM(P.Gender) LIKE "None"))
        GROUP BY CAST(SUBSTR(M.Year,-4) AS UNSIGNED)
        
        
        
        
        """

q5a = pd.read_sql_query(query, conn)    
print(q5a.shape)
q5a.head()

## Q5.b --- Now include a small change: report for each year the percentage of movies in that year with only female actors, and the total number of movies made that year. For example, one answer will be: 1990 31.81 13522 meaning that in 1990 there were 13,522 movies, and 31.81% had only female actors. You do not need to round your answer.

In [54]:
%%time
# Write your sql query below

query = """

        SELECT T2.Year, T2.num_of_movies_per_year as "Number of Movies",
              IFNULL((T1."Number of Movies"*100.0)/T2.num_of_movies_per_year,0) AS "Percentage"
        FROM
            (SELECT SUBSTR(M.Year,-4) as Year, count(*) AS 'num_of_movies_per_year'
                   FROM Movie M
                   GROUP BY CAST(SUBSTR(Year,-4) AS UNSIGNED)) T2
            
            LEFT JOIN  (SELECT SUBSTR(M.Year,-4) as 'Year' , count(*) AS "Number of Movies" 
            FROM Movie M
            WHERE TRIM(M.MID) IN (SELECT TRIM(MC.MID)
                            FROM M_Cast MC
                            WHERE TRIM(MC.PID, " ") IN (SELECT TRIM(P.PID)
                                                        FROM Person P
                                                        WHERE P.Gender = "Female"))
            AND TRIM(M.MID) NOT IN (SELECT TRIM(MC.MID)
                            FROM M_Cast MC
                            WHERE TRIM(MC.PID, " ") IN (SELECT TRIM(P.PID)
                                                        FROM Person P
                                                        WHERE P.Gender = "Male"))
            GROUP BY CAST(SUBSTR(Year,-4) AS UNSIGNED)) T1 ON CAST(SUBSTR(T1.Year,-4) AS UNSIGNED) = CAST(SUBSTR(T2.Year,-4) AS UNSIGNED)
         
        
        """

q5b = pd.read_sql_query(query, conn)    
print(q5b.shape)
q5b.tail(20)

(78, 3)
CPU times: user 150 ms, sys: 6.55 ms, total: 157 ms
Wall time: 156 ms


,Year,Number of Movies,Percentage
58,1999,66,1.515152
59,2000,64,1.562500
60,2001,73,0.000000
61,2002,87,0.000000
62,2003,103,0.000000
63,2004,103,0.000000
64,2005,129,0.000000
65,2006,101,0.000000
66,2007,109,0.000000
67,2008,107,0.000000


## Q6 --- Find the film(s) with the largest cast. Return the movie title and the size of the cast. By "cast size" we mean the number of distinct actors that played in that movie: if an actor played multiple roles, or if it simply occurs multiple times in casts, we still count her/him only once.

In [89]:
%%time
# Write your sql query below

query = """
        SELECT M.title, T1.Num_Of_Actors 
        FROM Movie M
        INNER JOIN (SELECT MC.MID, COUNT(DISTINCT MC.PID) AS "Num_Of_Actors"
                   FROM M_Cast MC
                   GROUP BY MC.MID
                   ORDER BY Num_Of_Actors DESC
                   LIMIT 1) T1 ON M.MID = T1.MID
        
        """

q6 = pd.read_sql_query(query, conn)    
print(q6.shape)
q6.head()

(1, 2)
CPU times: user 58.7 ms, sys: 12 ms, total: 70.8 ms
Wall time: 69.5 ms


,title,Num_Of_Actors
0,Ocean's Eight,238


## Q7 --- A decade is a sequence of 10 consecutive years. For example, say in your database you have movie information starting from 1965. Then the first decade is 1965, 1966, ..., 1974; the second one is 1967, 1968, ..., 1976 and so on. Find the decade D with the largest number of films and the total number of films in D.

In [66]:
%%time
# Write your sql query below

## source: https://stackoverflow.com/questions/51609285/query-for-find-the-decade-with-the-largest-number-of-records
## source: https://stackoverflow.com/questions/26518526/conditional-join-statement-sql-server

query = """
        SELECT CAST(SUBSTR(M.Year,-4) AS UNSIGNED) as "Decade Start", CAST(SUBSTR(M.Year,-4) AS UNSIGNED) + 9 as "Decade End",count(*) as "Num Of Movies"
        FROM (select distinct year from Movie) M 
        JOIN Movie M1 ON CAST(SUBSTR(M1.Year,-4) AS UNSIGNED) >= CAST(SUBSTR(M.Year,-4) AS UNSIGNED) 
                         and CAST(SUBSTR(M1.Year,-4) AS UNSIGNED) < CAST(SUBSTR(M.Year,-4) AS UNSIGNED) + 10
        GROUP BY M.year
        ORDER BY count(*) desc
        LIMIT 1;
        
        """

q7 = pd.read_sql_query(query, conn)    
print(q7.shape)
q7.head()

(1, 3)
CPU times: user 162 ms, sys: 3.02 ms, total: 165 ms
Wall time: 164 ms


,Decade Start,Decade End,Num Of Movies
0,2008,2017,1205


## Q8 --- Find all the actors that made more movies with Yash Chopra than any other director.

In [27]:
%%time
# Write your sql query below

query = """
    SELECT *
    FROM(
        SELECT T1.Actor, T1.Director, T1.Count_Director, RANK() OVER (PARTITION BY T1.Director ORDER BY T1.Count_Director DESC) as ranking
        FROM(
            SELECT distinct Casts.Actor, Directedby.Director, 
                   COUNT(Directedby.Director) OVER(PARTITION BY Casts.Actor) AS "Count_Director"
            FROM (SELECT MC.MID, TRIM(P.Name, " ") AS "Actor"
                  FROM M_Cast MC
                  JOIN Person P ON TRIM(MC.PID, " ") = P.PID) Casts
            JOIN (SELECT MD.MID, TRIM(P.Name, " ") AS "Director"
                  FROM M_Director MD
                  JOIN Person P ON TRIM(MD.PID, " ") = P.PID) DirectedBy ON Casts.MID = Directedby.MID
            GROUP BY Casts.Actor) T1)T2
    WHERE T2.Director = "Yash Chopra" 
        """

q8 = pd.read_sql_query(query, conn)    
print(q8.shape)
q8.head(100)

(163, 4)
CPU times: user 518 ms, sys: 17.2 ms, total: 535 ms
Wall time: 535 ms


,Actor,Director,Count_Director,ranking
0,Abbie Murphy,Yash Chopra,1,1
1,Akhtar Mirza,Yash Chopra,1,1
2,Akhtar-Ul-Iman,Yash Chopra,1,1
3,Amarinder Sodhi,Yash Chopra,1,1
4,Amir Zadey,Yash Chopra,1,1
...,...,...,...,...
95,Nigel Genis,Yash Chopra,1,1
96,Nirupa Roy,Yash Chopra,1,1
97,Om Sahni,Yash Chopra,1,1
98,Pamela Betsy Cooper,Yash Chopra,1,1


In [32]:
%%time
# Write your sql query below

query = """
        SELECT  Casts.MID,count(Casts.MID) OVER (PARTITION BY Directedby.Director, Casts.Actor) as "count_of_movies", Casts.Actor, Directedby.Director
        FROM (SELECT MC.MID, TRIM(P.Name, " ") AS "Actor"
             FROM M_Cast MC
             JOIN Person P ON TRIM(MC.PID, " ") = P.PID) Casts
        NATURAL JOIN (SELECT MD.MID, TRIM(P.Name, " ") AS "Director"
              FROM M_Director MD
             JOIN Person P ON TRIM(MD.PID, " ") = P.PID) DirectedBy 
        """

q8 = pd.read_sql_query(query, conn)    
print(q8.shape)
q8.head(100)

(82835, 4)
CPU times: user 703 ms, sys: 39.7 ms, total: 743 ms
Wall time: 746 ms


,MID,count_of_movies,Actor,Director
0,tt0156745,1,Abhimanyyu Abhimanyu,A. Bhimsingh
1,tt0156651,1,Achala Sachdev,A. Bhimsingh
2,tt0062640,1,Agha,A. Bhimsingh
3,tt0156651,1,Alka,A. Bhimsingh
4,tt0156833,1,Anand,A. Bhimsingh
...,...,...,...,...
95,tt0063533,4,Mukri,A. Bhimsingh
96,tt0156833,4,Mukri,A. Bhimsingh
97,tt0156745,4,Mukri,A. Bhimsingh
98,tt0059354,3,Mumtaz,A. Bhimsingh


In [62]:
%%time
# Write your sql query below

query = """

    WITH YashChopraCount AS(
        SELECT DISTINCT T2.Actor, T2.CountOfMovies
        FROM(
            SELECT DISTINCT TRIM(Casts.MID) , TRIM(Casts.Actor) as Actor,  TRIM(Directedby.Director) as Director, 
            COUNT(Casts.MID) over (partition by Casts.Actor,Directedby.Director) as CountOfMovies
            FROM (SELECT MC.MID, TRIM(P.Name, " ") AS "Actor"
                 FROM M_Cast MC
                 JOIN Person P ON TRIM(MC.PID, " ") = P.PID) Casts
            NATURAL JOIN (SELECT MD.MID, TRIM(P.Name, " ") AS "Director"
                  FROM M_Director MD
                  JOIN Person P ON TRIM(MD.PID, " ") = P.PID
                  WHERE P.Name LIKE "%Yash Chopra%") DirectedBy 
            ORDER BY Casts.Actor) T2 )

   

    SELECT distinct MCC.Actor
    FROM (SELECT distinct T1.Actor, MAX(T1.CountOfMoviesByDirector) OVER (PARTITION BY (T1.Actor)) as 'MaxMovie'
    FROM (
        SELECT distinct Casts.MID, Casts.Actor,  Directedby.Director, COUNT(Casts.MID) over (partition by Casts.Actor,Directedby.Director) as CountOfMoviesByDirector
        FROM (SELECT TRIM(MC.MID) AS MID, TRIM(P.Name, " ") AS "Actor"
             FROM M_Cast MC
             JOIN Person P ON TRIM(MC.PID, " ") = P.PID) Casts
        NATURAL JOIN (SELECT MD.MID, TRIM(P.Name, " ") AS "Director"
              FROM M_Director MD
             JOIN Person P ON TRIM(MD.PID, " ") = P.PID) DirectedBy 
        ORDER BY Casts.Actor) T1) MCC
    INNER JOIN YashChopraCount YCC ON MCC.Actor = YCC.Actor AND MCC.MaxMovie = YCC.CountOfMovies

     
    
    
    
        """

q8 = pd.read_sql_query(query, conn)    
print(q8.shape)
q8.head(100)

(237, 1)
CPU times: user 1.11 s, sys: 77.4 ms, total: 1.19 s
Wall time: 1.19 s


,Actor
0,Abbie Murphy
1,Achala Sachdev
2,Akhtar Mirza
3,Akhtar-Ul-Iman
4,Aloka Mukherjee
...,...
95,Leela Chitnis
96,Lucy Phelps
97,Magnus Emaka Einang
98,Mahan Swadesh


## Q9 --- The Shahrukh number of an actor is the length of the shortest path between the actor and Shahrukh Khan in the "co-acting" graph. That is, Shahrukh Khan has Shahrukh number 0; all actors who acted in the same film as Shahrukh have Shahrukh number 1; all actors who acted in the same film as some actor with Shahrukh number 1 have Shahrukh number 2, etc. Return all actors whose Shahrukh number is 2.

In [5]:
%%time
# Write your sql query below

query = """

SELECT P.Name
FROM Person P
WHERE TRIM(P.PID) IN(
        SELECT DISTINCT TRIM(MC.PID)
        FROM M_Cast MC
        WHERE TRIM(MC.MID) IN(
                SELECT DISTINCT TRIM(MC.MID) 
                FROM M_Cast MC
                WHERE TRIM(MC.PID) IN(
                    SELECT DISTINCT TRIM(MC.PID) AS Shahrukh_1
                    FROM M_Cast MC
                    WHERE TRIM(MC.MID) IN (
                            SELECT DISTINCT TRIM(MC.MID)
                            FROM M_Cast MC
                            WHERE TRIM(MC.PID) IN (
                                SELECT DISTINCT TRIM(P.PID) PID FROM Person P WHERE TRIM(P.Name) LIKE "%Shah rukh%"))))
        AND TRIM(MC.PID) NOT IN(SELECT DISTINCT TRIM(MC.PID) AS Shahrukh_1
                FROM M_Cast MC
                WHERE TRIM(MC.MID) IN (
                        SELECT DISTINCT TRIM(MC.MID)
                        FROM M_Cast MC
                        WHERE TRIM(MC.PID) IN 
                              (SELECT DISTINCT TRIM(P.PID) PID FROM Person P WHERE TRIM(P.Name) LIKE "%Shah rukh%"))))

        """

q9 = pd.read_sql_query(query, conn)    
print(q9.shape)
q9.head()

(25698, 1)
CPU times: user 253 ms, sys: 12 ms, total: 265 ms
Wall time: 266 ms


,Name
0,Freida Pinto
1,Rohan Chand
2,Damian Young
3,Waris Ahluwalia
4,Caroline Christl Long


In [18]:
%%time
# Write your sql query below

query = """
    
    SELECT DISTINCT TRIM(MC.PID)
    FROM M_Cast MC
    WHERE TRIM(MC.MID) IN(
        SELECT DISTINCT TRIM(MC.MID)
        FROM M_CAST MC
        WHERE TRIM(MC.PID) IN(
            SELECT DISTINCT TRIM(MC.PID)
            FROM M_CAST MC
            WHERE TRIM(MC.MID) IN (
                SELECT DISTINCT TRIM(MC.MID)
                FROM M_Cast MC
                WHERE TRIM(MC.PID) IN(
                    SELECT DISTINCT TRIM(P.PID)
                    FROM Person P
                    WHERE TRIM(P.Name) LIKE "%Shah Rukh Khan%"))
            AND TRIM(MC.PID) NOT IN(
                SELECT TRIM(P.PID)
                FROM Person P
                WHERE TRIM(P.Name) LIKE "%Shah Rukh Khan%")))
    AND TRIM(MC.PID) NOT IN (
        SELECT TRIM(P.PID)
        FROM Person P
        WHERE TRIM(P.Name) LIKE "%Shah Rukh Khan%")
    AND TRIM(MC.PID) NOT IN(
        SELECT DISTINCT TRIM(MC.PID)
            FROM M_CAST MC
            WHERE TRIM(MC.MID) IN (
                SELECT DISTINCT TRIM(MC.MID)
                FROM M_Cast MC
                WHERE TRIM(MC.PID) IN(
                    SELECT DISTINCT TRIM(P.PID)
                    FROM Person P
                    WHERE TRIM(P.Name) LIKE "%Shah Rukh Khan%"))
            AND TRIM(MC.PID) NOT IN(
                SELECT TRIM(P.PID)
                FROM Person P
                WHERE TRIM(P.Name) LIKE "%Shah Rukh Khan%"))
        """

q9 = pd.read_sql_query(query, conn)    
print(q9.shape)
q9.head()

(25698, 1)
CPU times: user 309 ms, sys: 17.1 ms, total: 326 ms
Wall time: 325 ms


,TRIM(MC.PID)
0,nm2539953
1,nm0922035
2,nm0324658
3,nm0943079
4,nm0000218
